## Installing libraries

In [1]:
# !pip install cohere
# !pip install langchain
# !pip install pypdf
# !pip install tiktoken
# !pip install chromadb
# !pip install bs4

  Using cached pydantic-2.7.0-py3-none-any.whl.metadata (103 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.18.1-cp312-none-win_amd64.whl.metadata (6.7 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/150.3 kB ? eta -:--:--
   ---------------------------------------- 150.3/150.3 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/487.4 kB ? eta -:--:--
   --------------------------------------- 487.4/487.4 kB 10.4 MB/s eta 

## read url source file

In [2]:
def read_urls(file_path):
    '''Function to read URLs from a file'''
    with open(file_path, 'r') as file:
        urls = [url.strip() for url in file.readlines()]
    return urls

urls = read_urls("./source_url.txt")

## load the url source

In [3]:
from langchain.document_loaders import WebBaseLoader

# Specify the URL of the website you want to load
all_documents = []

# Load the website content for each URL and add it to the list
for url in urls:
    loader = WebBaseLoader(url)
    documents = loader.load()
    all_documents.extend(documents)

all_documents

[Document(page_content='\n405 Not Allowed\n\n405 Not Allowed\nAngie\n\n\n', metadata={'source': 'https://www.mql5.com/en/blogs/post/752096', 'title': '405 Not Allowed', 'language': 'No language found.'}),
 Document(page_content='\n405 Not Allowed\n\n405 Not Allowed\nAngie\n\n\n', metadata={'source': 'https://www.mql5.com/en/blogs/post/751762', 'title': '405 Not Allowed', 'language': 'No language found.'})]

## load document from local file

In [4]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./source.txt')
all_documents = loader.load()

all_documents

[Document(page_content='Trades:\n744\nProfit Trades:\n490 (65.86%)\nLoss Trades:\n254 (34.14%)\nBest trade:\n57.45 USD\nWorst trade:\n-45.59 USD\nGross Profit:\n3 246.73 USD (344 065 pips)\nGross Loss:\n-1 705.90 USD (159 746 pips)\nMaximum consecutive wins:\n43 (101.14 USD)\nMaximal consecutive profit:\n124.46 USD (4)\nSharpe Ratio:\n0.19\nTrading activity:\n100.00%\nMax deposit load:\n8.24%\nLatest trade:\n3 days ago\nTrades per week:\n15\nAvg holding time:\n7 days \nRecovery Factor:\n12.02\nLong Trades:\n354 (47.58%)\nShort Trades:\n390 (52.42%)\nProfit Factor:\n1.90\nExpected Payoff:\n2.07 USD\nAverage Profit:\n6.63 USD\nAverage Loss:\n-6.72 USD\nMaximum consecutive losses:\n8 (-34.22 USD)\nMaximal consecutive loss:\n-75.82 USD (6)\nMonthly growth:\n5.89%\nAnnual Forecast:\n72.88%\nAlgo trading:\n73%\n\nGeneral Questions \nHow do I install expert advisors or indicators from the mql5.com market?\n\nKindly read my installation guide.\n\nHow do I update expert advisors or indicators f

## split the website content into smaller chunks

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os
import cohere

key = "urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg"
co = cohere.Client(key)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

os.environ["COHERE_API_KEY"] = key

## store vector

In [6]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(cohere_api_key=key)
db = Chroma.from_documents(docs, embeddings)

c:\Users\Admin\miniconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(


## define prompt template

In [7]:
from langchain.prompts import PromptTemplate
prompt_template = """
you are a personal financial advisor here to help the client. 

Given the following financial context:

{context}

Question: {question}
As your financial advisor, I would recommend the following:
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

## retrieve dara and output

In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import Cohere

retriever = db.as_retriever()  # Assuming 'db' is your vector store

retrievalQA = RetrievalQA.from_llm(
    llm=Cohere(),
    retriever=retriever,
    prompt=prompt
)

c:\Users\Admin\miniconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


## query 

In [9]:
query = "How do I update expert advisors or indicators from the mql5.com market to latest versions?"
retrievalQA.invoke(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'How do I update expert advisors or indicators from the mql5.com market to latest versions?',
 'result': ' To update an expert advisor or indicator from the mql5.com market, you can follow these enhanced steps:\n\n1. Log in to your MT5 account and select the "Market" option in the upper menu.\n2. On the market dashboard, click on the "Updates" icon beside the search bar.\n3. A list of available updates will appear. Go through the list and identify the expert advisors or indicators you wish to update.\n4. Tick the respective products and click the "Update" button at the bottom of the window.\n5. A pop-up notification will appear, confirming the update process. Follow the on-screen prompts to complete the update.\n\nIt\'s important to note that updates are free, and you can download the latest versions of the products you\'ve previously purchased from the market. Keep in mind that the "Update" button will only be available if a update is ready for a product. \n\nIf you still fa

In [10]:
query = "can you summarize my trading performance?"
retrievalQA.invoke(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'can you summarize my trading performance?',
 'result': " Sure! Here's a summary of your trading performance: \n\nYou have experienced a total of 744 trades, 490 of which were profitable, resulting in a 65.86% profit ratio. Additionally, your best trade yielded a profit of $57.45, while the worst trade resulted in a loss of -$45.59. Your gross profit amounted to $3,246.73, while your gross loss was -$1,705.90. Consecutive wins and consecutive losses ranged from 43 trades and 101.14 USD of profit, to a maximum consecutive loss of -34.22 USD, indicating some variance and risk in your trading activity. It is important to note that the provided context indicates 100.00% trading activity and an 8.24% maximum deposit load. The latest trade took place 3 days ago and you typically engage in 15 trades per week, with an average holding time of 7 days. Your trading strategy showcases a Recovery Factor of 12.02, a Profit Factor of 1.90, and an Expected Payoff of 2.07 USD. Your average pr

## invalid query

In [11]:
invalid_query = "where can i buy icecream?"
retrievalQA.invoke(invalid_query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'query': 'where can i buy icecream?',
 'result': ' I appreciate your kind reminder to read the installation guide for updating expert advisors and indicators on mql5.com. \n\nNow, regarding your query about buying ice cream, I interpret this question as a reflection of your interest in exploring additional personal investments with a recreational twist. As we speak, there are myriad avenues for you to explore in the food industry, and ice cream stands as a versatile and indulgent choice in this spectrum. \n\nBelow are viable pathways to harness the financial opportunities within the realm of ice cream: \n\n1. Local Ice Cream Shop Investor: Investing in a local ice cream shop can be an attractive opportunity to foster financial growth while nurturing your community. The shop provides an experiential product that inherently has high demand across different demographics. Moreover, it opens avenues to network with fellow entrepreneurs and build meaningful connections with the local custom

In [1]:
data = [0,0,2,7.4,4,-1.46,6,-2.85,8,-0.14,10,12.98,12,4.4,14,18.23,16,46.08,18,59.02,20,73.04,22,64.7,24,77.65,26,81.23,28,87.73,30,96.92,32,101.94,34,121.21,36,137.65,38,147.23,40,157.76,42,170.14,44,157.08,46,195.42,48,196.14,50,192.14,52,183.83,54,197.11,56,202.37,58,216.39,60,221.2,62,225.89,64,231.96,66,263.6,68,236.45,70,260.45,72,275.01,74,262.93,76,279.54,78,294.02,80,284.41,82,309.94,84,364.04,86,385.17,88,390.31,90,353.17,92,406.13,94,435.14,96,403.24,98,441.36,100,435.34,102,418.22,104,392.7,106,445.85,108,405.94,110,392.82,112,419.8,114,471.21,116,452.3,118,528.06,120,491.47,122,484.21,124,483.68,126,534.18,128,584.42,130,583.72,132,586.44,134,588.3,136,597.39,138,610.48,140,627.9,142,633.7,144,638.1,146,645.89,148,650.09,150,653.1,152,656.87,154,660.72,156,668.24,158,680.25,160,688.13,162,695.44,164,698.48,166,701.32,168,712.58,170,725.49,172,731.85,174,735.23,176,744.99,178,750.29,180,753.27,182,764.53,184,766.05,186,781.59,189,796.82,191,818.03,193,818.33,195,815.87,197,855.32,199,835.08,201,823.43,203,812.71,205,818.99,207,829.67,209,815.66,211,816.22,213,819.27,215,823.37,217,834.69,219,819.6,221,821.93,223,823.01,225,838.76,227,853.82,229,880.14,231,908.78,233,946.85,235,958.32,237,979.82,239,980.02,241,1012.03,243,1020.78,245,1019.21,247,1000.93,249,988.31,251,972.78,253,1015.16,255,997.96,257,1020.47,259,1004.1,261,1014.75,263,1042.48,265,1079.06,267,1105.21,269,1070.52,271,1047.96,273,1071.53,275,1095.64,277,1121.31,279,1154.55,281,1143.26,283,1143.72,285,1187.68,287,1274.83,289,1272.32,291,1268.56,293,1263.58,295,1258.83,297,1260.58,299,1226.62,301,1211.08,303,1196.59,305,1187.96,307,1168.96,309,1142.5,311,1139.45,313,1145.78,315,1146.82,317,1140.08,319,1140.49,321,1146.03,323,1145.98,325,1149.62,327,1139.93,329,1148.28,331,1144.33,333,1143.09,335,1151.3,337,1159.08,339,1160.19,341,1152.65,343,1177.77,345,1146.95,347,1144.91,349,1168.93,351,1159.72,353,1159.18,355,1158.99,357,1160.65,359,1161.45,361,1163.93,363,1162.86,365,1166.03,367,1169.05,369,1182.11,371,1169.59,373,1194.09,375,1197.8,377,1231.13,379,1233.23,381,1253.63,383,1233.73,385,1252.48,387,1252.11,389,1254.56,391,1285.42,393,1388.23,395,1354.44,397,1332.14,399,1384.33,401,1403.78,403,1432.28,405,1432.88,407,1442.99,409,1436.74,411,1446.08,413,1449.72,415,1453.02,417,1456.27,419,1463.56,421,1469.3,423,1467.8,425,1469.57,427,1468.36,429,1470.35,431,1472.35,433,1474.09,435,1477.06,437,1493.26,439,1483.9,441,1485.89,443,1488.88,445,1503.98,447,1498.5,449,1510.54,451,1516.1,453,1530.23,455,1546.29,457,1543.64,459,1560.11,461,1562.39,463,1577.88,465,1581.49,467,1588.72,469,1588.34,471,1592.39,473,1596.82,475,1597.34,477,1598.94,479,1603.17,481,1604.71,483,1606.06,485,1611,487,1620.44,489,1619.99,491,1620.14,493,1621.33,495,1623.53,497,1623.31,499,1623.84,501,1629.8,503,1627.78,505,1631.73,507,1628.64,509,1629.04,511,1629.05,513,1631.73,515,1623.93,517,1620.85,519,1616.88,521,1617.89,523,1619.53,525,1628.69,527,1632.77,529,1627.32,531,1627.78,533,1631.64,535,1629.73,537,1627.12,539,1626.53,541,1625.01,543,1628,545,1625.55,547,1639.37,549,1632.89,551,1638.62,553,1643.78,555,1649.06,557,1648.29,559,1647.28,561,1649.59,563,1653.58,565,1661.64,567,1674.02,569,1669.45,571,1675.23,573,1683.45,575,1636.16,577,1637.84,579,1577.15,581,1563.52,583,1562.18,585,1563.51,587,1572.08,589,1608.95,591,1609.31,593,1614.58,595,1619.7,597,1626.3,599,1650.77,601,1650.92,603,1663.32,605,1655.93,607,1663.82,609,1671.75,611,1679.64,613,1684.01,615,1688.21,617,1673.8,619,1682.41,621,1687.71,623,1687.51,625,1702.18,627,1707.33,629,1720.43,631,1718.42,633,1721.36,635,1724.24,637,1732.41,639,1736.01,641,1739.88,643,1741.74,645,1745.73,647,1748.59,649,1764.05,651,1769.17,653,1771,655,1772.27,657,1773.51,659,1777.71,661,1780.17,663,1781.69,665,1785.26,667,1789.05,669,1792.45,671,1795.72,673,1799.33,675,1808.71,677,1818.92,679,1839.7,681,1844.45,683,1856.01,685,1868.62,687,1867.97,689,1869.27,691,1871.92,693,1894.43,695,1910.84,697,1915.22,699,1912.58,701,1938.41,703,1937.53,705,1943.17,707,1951.84,709,1969.87,711,1965.91,713,1967.7,715,1967.58,717,1983.56,719,1990.48,721,2001.65,723,2007.74,725,2012.7,727,2019.4,729,2025.64,731,2032.02,733,2041.79,735,2046.02,737,2053.07,739,2058.41,741,2064.7,743,2072.79,745,2077.52,747,2079.68,749,2033.87]

In [3]:
import csv

# Divide data by 100
data = [x / 100 for x in data]

# Create a list of tuples for each row in the CSV
rows = [(i // 2 + 1, data[i]) for i in range(len(data)) if i % 2 == 0]

# Define the CSV filename
filename = 'growth_data.csv'

# Write data to CSV file
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['trade_number', 'growth'])  # Write header
    writer.writerows(rows)  # Write data rows

print(f"Data has been successfully formatted and saved as '{filename}'.")

Data has been successfully formatted and saved as 'growth_data.csv'.
